In [ ]:
import pandas as pd
import pyodbc
from datetime import datetime 
from sqlalchemy import create_engine

# odczytuję dane z pliku csv 
economist_country_codes = pd.read_csv('data/economist_country_codes0.csv',sep=';|\|',engine='python')
economist_country_codes = economist_country_codes.sort_values('COUNTRY',ignore_index=True) # posortowane lepiej wygląda

# dodaję potrzebne kolumny zdefiniowane 
economist_country_codes['geography_id'] = economist_country_codes.index+1 # chcę żeby geography_id zaczynało się od 1
economist_country_codes['valid_from'] = datetime.today().strftime('%Y-%m-%d')
economist_country_codes['valid_to'] = '9999-01-01'
economist_country_codes['active'] = 1 

# zmiana nazw i kolejności kolum żeby pasowały do nadanych w docelowej tabeli
economist_country_codes.columns=['country_name', 'country_code', 'region', 'geography_id', 'valid_from', 'valid_to','active']
economist_country_codes = economist_country_codes[['geography_id','country_code','country_name','region','valid_from','valid_to','active']]

# ustanawiam połączenie z bazą
SERVER = 'bigmacindex.database.windows.net'
DATABASE = 'bigmacindex'
DRIVER = 'ODBC Driver 17 for SQL Server'

# docelowo folder z credentialami nie byłby ogólno dostępny, ale chciałem żeby skrypt można było uruchomić 
with open('creds/data_writer_creds.txt','r') as f:
    creds = [line.strip() for line in f.readlines()]

USERNAME = creds[0] 
PASSWORD = creds[1] 


DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()


# ładuje dane do tabeli geographies
economist_country_codes.to_sql(name='geographies',con=connection,if_exists='append', index=False)

connection.close()